# Generate Data

In [1]:
# Imports
from pathlib import Path
from tqdm import tqdm
from elements import elements
from mendeleev import element

from Code.cifSimulation import structureGenerator
from Code.h5Constructor import h5Constructor

import cProfile

## Cif generation

In [ ]:
# Path to dataset
datasetPath = './Dataset/CIFs/SimulatedTest2/'

# Initialize the CIF generator
generator = structureGenerator()

# Maximum number of atom species in generated structures
n_species = 2

# Choose required atoms to use
required_atoms = ['O']

# Choose optional atoms to use
metals = [atom.Symbol for atom in elements.Alkali_Metals] 
metals += [atom.Symbol for atom in elements.Alkaline_Earth_Metals] 
metals += [atom.Symbol for atom in elements.Transition_Metals] 
metals += [atom.Symbol for atom in elements.Metalloids] 
metals += [atom.Symbol for atom in elements.Others]

optional_atoms = []
for metal in metals:
    try:
        elm_data = element(metal)
        if elm_data.metallic_radius or elm_data.atomic_radius:
            optional_atoms.append(metal)
        else:
            print(f'Removed {metal} from dataset as no table values were available.')
    except:
        print(f'Removed {metal} from dataset as no table values were available.')
optional_atoms = ['Zn', 'Nb']

# Simulate mono-metal oxides
generator.create_cif_dataset(
    n_species=n_species,
    required_atoms=required_atoms,
    optional_atoms=optional_atoms,
    from_table_values=False,
    save_folder=datasetPath,
)

## Query COD

In [ ]:
from pathlib import Path
import requests
import zipfile
import io
from multiprocessing import Pool, cpu_count
from itertools import islice, repeat
from tqdm.auto import tqdm
import argparse
from elements import elements
from mendeleev import element
import pandas as pd

In [ ]:
metals = [atom.Symbol for atom in elements.Alkali_Metals] 
metals += [atom.Symbol for atom in elements.Alkaline_Earth_Metals] 
metals += [atom.Symbol for atom in elements.Transition_Metals] 
metals += [atom.Symbol for atom in elements.Metalloids] 
metals += [atom.Symbol for atom in elements.Others] # Post-transition metals
metals += ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb',
           'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu'] # Lanthanides

# Remove elements that does not have a well defined radius or are rare in nanoparticles
unwanted_elements = ['Fr', 'Po', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Uub', 'Uun', 'Uuu']
for elm in unwanted_elements:
    metals.remove(elm)
    
print(len(metals))

In [ ]:
non_metals = [atom.Symbol for atom in elements.Non_Metals]
non_metals += [atom.Symbol for atom in elements.Halogens]
non_metals.remove('At')

print(len(non_metals))

In [ ]:
# Find all possible two element combinations of metals and non-metals
combinations = []
for metal in metals:
    for non_metal in non_metals:
        combinations.append(([metal, non_metal], []))

for metal in metals:
    combinations.append(([metal], non_metals))

print(len(combinations))

In [ ]:
def queryCOD_new(input_tuple):
    (included_atoms, excluded_atoms), max_volume = input_tuple

    id_url = 'https://www.crystallography.net/cod/result?format=lst'
    if included_atoms:
        for i, included_atom in enumerate(included_atoms):
            id_url += f'&el{i+1}={included_atom}'
    if excluded_atoms:
        for i, excluded_atom in enumerate(excluded_atoms):
            id_url += f'&nel{i+1}={excluded_atom}'
    if max_volume:
        id_url += f'&vmax={max_volume}'
    
    id_response = requests.get(id_url)

    return id_response.text.split('\n')

In [ ]:
inputs = zip(combinations, repeat(1000))
id_list = []
with Pool(processes=cpu_count()-1) as pool:
    with tqdm(total=len(combinations), desc='Querying COD') as pbar:
        for returned_ids in pool.imap_unordered(queryCOD_new, inputs, chunksize=1):
            id_list.extend(returned_ids)
            pbar.update()

In [ ]:
df_ids = pd.DataFrame(id_list)
print(len(df_ids))
df_ids = df_ids.drop_duplicates()
print(len(df_ids))
df_ids.to_csv('./test_cif_IDs.csv', index=False, header=False)

In [ ]:
from Code.cifCleaning import cif_cleaning_pipeline
from Code.databaseQuery import queryCOD

# Path to dataset
cif_folder = './Dataset/CIFs/COD_subset/'

# Path to CIF ID list
id_list_path = './COD_subset_IDs.csv'

queryCOD(cif_folder, id_file=id_list_path)

In [ ]:
df_cifs = cif_cleaning_pipeline(cif_folder, chunksize=100)

In [ ]:
df_cifs.head()

In [ ]:
df_cifs[(df_cifs['Element0'] == 6) & (df_cifs['Element1'] == 8) & (df_cifs['Element2'].isna())]

In [ ]:
from ase.io import read
cif = read(df_cifs['filepath'].iloc[19477], format='cif')

In [ ]:
cif.symbols

In [ ]:
metals

In [ ]:
metal_numbers = [element(metal).atomic_number for metal in metals]
metal_numbers

In [ ]:
# df_cifs.columns.difference(['filepath', 'Spacegroup'])
contains_metals = df_cifs.apply(lambda row: any(atom in metal_numbers for atom in row[2:].values), axis=1)

In [ ]:
sum(~contains_metals)

In [ ]:
df_cifs[~contains_metals]

In [ ]:
# Cout nan values
df_cifs[df_cifs['Element1'].isna()].sort_values(by='filepath')

In [ ]:
from ase.io import read
from tqdm.auto import trange
metal_cifs = df_cifs[df_cifs['Element1'].isna()].sort_values(by='filepath')


In [ ]:
unit_cell = read(metal_cifs['filepath'].iloc[7].replace('_cleaned', ''))
print(unit_cell.get_chemical_formula())
print(unit_cell.get_chemical_symbols())
print(unit_cell.cell.cellpar())


In [ ]:
df_cifs.columns

In [ ]:
columns = ['Spacegroup', 'Element0', 'Element1', 'Element2',
       'Element3', 'Element4', 'Element5', 'Element6', 'Element7', 'Element8',
       'Element9', 'Element10']
for i in trange(len(df_cifs)):
    specific_row = df_cifs.iloc[i]

    duplicate_cifs = df_cifs[(df_cifs[columns] == specific_row[columns]).all(axis=1)]
    if len(duplicate_cifs) > 0:
        print(specific_row)
        print(duplicate_cifs.head())
        break



## .h5 generation

In [2]:
gc = h5Constructor(cif_dir='./Dataset/CIFs/SimulatedTest/', save_dir='./Dataset/h5/SimulatedTest2/')
# cProfile.run('gc.gen_h5s(parallelize=False, device="cuda")', sort='tottime')
gc.gen_h5s(parallelize=False, device='cuda', np_radii=[5, 10, 15, 20, 25])

Rutile_MoO2
NaCl_MoO
AntiFluorite_Co2O
Wurtzite_IrO
AntiFluorite_Pt2O
Wurtzite_CoO
ReO3_IrO3
Wurtzite_MoO
Spinel_FeFe2O4
CdI2_FeO2
AntiFluorite_Mo2O
Rutile_IrO2
Rutile_PtO2
ReO3_PtO3
AntiFluorite_Fe2O
NaCl_PtO
CdI2_CoO2
Wurtzite_PtO
NaCl_FeO
NaCl_IrO
Wurtzite_FeO
CdI2_MoO2
Fluorite_FeO2
Spinel_PtPt2O4
Spinel_MoMo2O4
Fluorite_CoO2
Fluorite_PtO2
Rutile_FeO2
CdI2_PtO2
ReO3_CoO3
Fluorite_MoO2
ReO3_MoO3
Spinel_IrIr2O4
AntiFluorite_Ir2O
ReO3_FeO3
Rutile_CoO2
NaCl_CoO
Spinel_CoCo2O4
Fluorite_IrO2
CdI2_IrO2


## Create unit cell graph

In [2]:
#%% Imports
import os, sys, math, torch
from mendeleev import element
from mendeleev.fetch import fetch_table
import numpy as np
from tqdm.auto import tqdm
import h5py
from itertools import repeat
from multiprocessing import Pool, cpu_count
from pathlib import Path
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from networkx.algorithms.components import is_connected
from ase.io import read
from ase.build import make_supercell
from ase.spacegroup import get_spacegroup
from ase.visualize import view
# from Code.simScatteringPatterns import simPDFs, cif_to_NP_GPU
from debyecalculator import DebyeCalculator
from elements import elements
from mendeleev import element
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.core.structure import Structure
from pymatgen.analysis.local_env import CrystalNN, MinimumDistanceNN
from pymatgen.io.ase import AseAtomsAdaptor
from timeit import default_timer as timer

In [3]:
# Path to cif file
# cif = './Dataset/CIFs/COD_subset_cleaned/1000023.cif'
#cif = './Dataset/CIFs/COD_subset_cleaned/1008022.cif'
cif = './Dataset/CIFs/COD_subset_cleaned/5910132.cif'

# Load cif
unit_cell = read(cif)

# Find spacegroup
spacegroup = get_spacegroup(unit_cell)

/home/ufj/anaconda3/envs/dataset_env/lib/python3.10/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(88, setting=1). This may result in wrong setting!
  warnings.warn(
/home/ufj/anaconda3/envs/dataset_env/lib/python3.10/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 0 and 2 are equivalent
  warnings.warn('scaled_positions %d and %d '
/home/ufj/anaconda3/envs/dataset_env/lib/python3.10/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 1 and 3 are equivalent
  warnings.warn('scaled_positions %d and %d '
/home/ufj/anaconda3/envs/dataset_env/lib/python3.10/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 1 and 5 are equivalent
  warnings.warn('scaled_positions %d and %d '
/home/ufj/anaconda3/envs/dataset_env/lib/python3.10/site-packages/ase/spacegroup/spacegroup.py:433: UserWarning: scaled_positions 4 and 6 are equivalent
  warnings.

In [4]:
unit_cell.get_all_distances(mic=True)

array([[0.        , 3.70130214, 3.70130214, ..., 5.74189944, 1.17578831,
        4.28343538],
       [3.70130214, 0.        , 6.85058574, ..., 2.72219005, 4.28343538,
        5.74189944],
       [3.70130214, 6.85058574, 0.        , ..., 4.28343538, 2.72219005,
        1.17578831],
       ...,
       [5.74189944, 2.72219005, 4.28343538, ..., 0.        , 6.3706946 ,
        3.24457236],
       [1.17578831, 4.28343538, 2.72219005, ..., 6.3706946 , 0.        ,
        3.24457236],
       [4.28343538, 5.74189944, 1.17578831, ..., 3.24457236, 3.24457236,
        0.        ]])

In [ ]:
unit_cell.get_chemical_symbols()

In [ ]:
unit_cell.get_scaled_positions()

In [ ]:
# view(unit_cell, viewer='ngl')

In [ ]:
node_feature_table = fetch_table('elements')[['atomic_number', 'atomic_radius', 'atomic_weight', 'electron_affinity']]
node_feature_table['electron_affinity'].fillna(0.0, inplace=True)

# Metals of interest
metals = [atom.Symbol for atom in elements.Alkali_Metals] 
metals += [atom.Symbol for atom in elements.Alkaline_Earth_Metals] 
metals += [atom.Symbol for atom in elements.Transition_Metals] 
metals += [atom.Symbol for atom in elements.Metalloids] 
metals += [atom.Symbol for atom in elements.Others] # Post-transition metals
metals += ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb',
        'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu'] # Lanthanides

# Remove elements that does not have a well defined radius or are rare in nanoparticles
unwanted_elements = ['Fr', 'Po', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Uub', 'Uun', 'Uuu']
for elm in unwanted_elements:
    metals.remove(elm)
    
# Convert to atomic numbers
metals = [element(metal).atomic_number for metal in metals]

In [ ]:
import plotly.graph_objects as go

def plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features):
    
    jmol_colors = [
        "#FFFFFF", "#8F8F8F", "#D9FFFF", "#CC80FF", "#C2FF00", "#FFB5B5", "#909090", "#3050F8", "#FF0D0D", "#90E050",
        "#B3E3F5", "#AB5CF2", "#8AFF00", "#BFA6A6", "#F0C8A0", "#FF8000", "#FFFF30", "#1FF01F", "#80D1E3", "#8F40D4",
        "#3DFF00", "#E6E6E6", "#BFC2C7", "#A6A6AB", "#8A99C7", "#9C7AC7", "#E06633", "#F090A0", "#50D050", "#C88033",
        "#7D80B0", "#C28F8F", "#668F8F", "#BD80E3", "#FFA100", "#A62929", "#5CB8D1", "#702EB0", "#00FF00", "#94FFFF",
        "#94E0E0", "#73C2C9", "#54B5B5", "#3B9E9E", "#248F8F", "#0A7D8C", "#006985", "#C0C0C0", "#FFD98F", "#A67573",
        "#668080", "#9E63B5", "#D47A00", "#940094", "#429EB0", "#57178F", "#00C900", "#70D4FF", "#FFFFC7", "#D9FFC7",
        "#C7FFC7", "#A3FFC7", "#8FFFC7", "#61FFC7", "#45FFC7", "#30FFC7", "#1FFFC7", "#00FF9C", "#00E675", "#00D452",
        "#00BF38", "#00AB24", "#4DC2FF", "#4DA6FF", "#2194D6", "#267DAB", "#266696", "#175487", "#D0D0E0", "#FFD123",
        "#B8B8D0", "#A6544D", "#575961", "#9E4FB5", "#AB5C00", "#754F45", "#428296", "#420066", "#007D00", "#70ABFA",
        "#00BAFF", "#00A1FF", "#008FFF", "#0080FF", "#006BFF", "#545CF2", "#785CE3", "#8A4FE3", "#A136D4", "#B31FD4",
        "#B31FBA", "#B30DA6", "#BD0D87", "#C70066", "#CC0059", "#D1004F", "#D90045", "#E00038", "#E6002E", "#EB0026",
        "#EB0026", "#F1001E"
    ]
    
    # Create a figure
    fig = go.Figure()
    
    # Add atoms to the figure
    for i in range(len(node_pos_real)):
        atom = node_pos_real[i]
        fig.add_trace(go.Scatter3d(
            x=[atom[0]],
            y=[atom[1]],
            z=[atom[2]],
            mode='markers',
            marker=dict(
                size=5,
                color=jmol_colors[int(node_features[i][0])],
            ),
            name=f'Atom {i+1}\n({node_features[i][0]})'
        ))

    # Add bonds to the figure
    for i in range(len(direction[0])):
        atom1 = node_pos_real[direction[0][i]]
        atom2 = node_pos_real[direction[1][i]]
        fig.add_trace(go.Scatter3d(
            x=[atom1[0], atom2[0]],
            y=[atom1[1], atom2[1]],
            z=[atom1[2], atom2[2]],
            mode='lines',
            line=dict(
                color='black',
                width=2
            ),
            name=f'Bond {i+1}\n({edge_features[i]:.2f})'
        ))

    # Set layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        ),
        title='Atom Positions and Bonds',
        showlegend=True
    )

    # Show the figure
    fig.show()

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
unit_cell_dist = unit_cell.get_all_distances(mic=True)
unit_cell_atoms = unit_cell.get_atomic_numbers().reshape(-1, 1)

# Make supercell to get Lattice constant
supercell = make_supercell(unit_cell, np.diag([2,2,2]))
# supercell = unit_cell
metal_distances = supercell[supercell.get_atomic_numbers() != 8].get_all_distances()
lc = np.amin(metal_distances[metal_distances > 0.])

# Create edges and node features
lc_mask = (unit_cell_dist > 0) & (unit_cell_dist < lc)
oxy_mask = np.outer(unit_cell_atoms == 8, unit_cell_atoms == 8)
metal_mask = np.outer(unit_cell_atoms != 8, unit_cell_atoms !=8)
direction = np.argwhere(lc_mask & ~oxy_mask & ~metal_mask).T

edge_features = unit_cell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]-1].values
    for atom in unit_cell_atoms
    ], dtype='float')
node_pos_real = unit_cell.get_positions()
node_pos_relative = unit_cell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
# plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features)

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
unit_cell_dist = unit_cell.get_all_distances(mic=True)
unit_cell_atoms = unit_cell.get_atomic_numbers().reshape(-1, 1)

# metal_filter = np.isin(unit_cell_atoms, metals)

# Find node features
node_features = np.array([
    node_feature_table.loc[atom[0]-1].values
    for atom in unit_cell_atoms
    ], dtype='float')

# Create mask of threshold for bonds
bond_threshold = np.zeros_like(unit_cell_dist)
for i, r1 in enumerate(node_features[:,1]):
    bond_threshold[i,:] = (r1 + node_features[:,1]) / 100 * 1.25
np.fill_diagonal(bond_threshold, 0.)

# Find edges
direction = np.argwhere(unit_cell_dist < bond_threshold).T

edge_features = unit_cell_dist[direction[0], direction[1]]

node_pos_real = unit_cell.get_positions()
node_pos_relative = unit_cell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
# plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features)

In [ ]:
unit_cell_atoms.T

In [ ]:
direction

In [ ]:
node_pos_relative

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
unit_cell_dist = unit_cell.get_all_distances(mic=True)
unit_cell_atoms = unit_cell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(unit_cell)
crystal_nn = CrystalNN()
strucGraph = StructureGraph.with_local_env_strategy(struc, crystal_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = unit_cell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]-1].values
    for atom in unit_cell_atoms
    ], dtype='float')
node_pos_real = unit_cell.get_positions()
node_pos_relative = unit_cell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
# plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features)

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
unit_cell_dist = unit_cell.get_all_distances(mic=True)
unit_cell_atoms = unit_cell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(unit_cell)
crystal_nn = CrystalNN(distance_cutoffs=None, x_diff_weight=0.0, porous_adjustment=False)
strucGraph = StructureGraph.with_local_env_strategy(struc, crystal_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = unit_cell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]].values
    for atom in unit_cell_atoms
    ], dtype='float')
node_pos_real = unit_cell.get_positions()
node_pos_relative = unit_cell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
# plot_atoms_and_bonds(node_pos_real, node_features, direction)

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
unit_cell_dist = unit_cell.get_all_distances(mic=True)
unit_cell_atoms = unit_cell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(unit_cell)
minimum_nn = MinimumDistanceNN()
strucGraph = StructureGraph.with_local_env_strategy(struc, minimum_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = unit_cell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]-1].values
    for atom in unit_cell_atoms
    ], dtype='float')
node_pos_real = unit_cell.get_positions()
node_pos_relative = unit_cell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
# plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features)

## Create supercell graph

In [ ]:
r_max = 10 # Maximum radius of the particle
cell_dims = unit_cell.cell.cellpar()[:3] # Get the cell dimensions
padding = 2 # Symmetric padding to ensure the particle does not exceed the supercell boundary
supercell_matrix = np.diag((np.ceil(r_max / cell_dims)) * 2 + padding)
supercell = make_supercell(prim=unit_cell, P=supercell_matrix)
# supercell = make_supercell(unit_cell, np.diag([2,2,2]))
# supercell.center(about=0)
supercell.get_positions().max(axis=0)

In [ ]:
cell_dims

In [ ]:
unit_cell.get_positions().max(axis=0)

In [ ]:
view(supercell, viewer='ngl')

In [ ]:
from pymatgen.analysis.local_env import CrystalNN, MinimumDistanceNN, IsayevNN, JmolNN, MinimumOKeeffeNN, MinimumVIRENN, OpenBabelNN, VoronoiNN

In [ ]:
# Setup of structure and local neighborhood strategies
struc = AseAtomsAdaptor.get_structure(supercell)
crystal_nn = CrystalNN()
minDist_nn = MinimumDistanceNN()
isayev_nn = IsayevNN()
jmol_nn = JmolNN()
keeffe_nn = MinimumOKeeffeNN()
viree_nn = MinimumVIRENN()
voronoi_nn = VoronoiNN()

In [ ]:
# CrystalNN
start_crystalNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, crystal_nn)
stop_crystalNN = timer()
# Save number of edges
nEdges_crystalNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]


In [ ]:
# MinimumDistanceNN
start_minDistNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, minDist_nn)
stop_minDistNN = timer()
# Save number of edges
nEdges_minDistNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
# IsayevNN
start_isayevNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, isayev_nn)
stop_isayevNN = timer()
# Save number of edges
nEdges_isayevNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
# JmolNN
start_jmolNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, jmol_nn)
stop_jmolNN = timer()
# Save number of edges
nEdges_jmolNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
# MinimumOKeeffeNN
start_okeeffeNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, keeffe_nn)
stop_okeeffeNN = timer()
# Save number of edges
nEdges_okeeffeNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
# MinimumVIRENN
start_vireeNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, viree_nn)
stop_vireeNN = timer()
# Save number of edges
nEdges_vireeNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
# VoronoiNN
start_voronoiNN = timer()
# Find edges
strucGraph = StructureGraph.with_local_env_strategy(struc, voronoi_nn)
stop_voronoiNN = timer()
# Save number of edges
nEdges_voronoiNN = np.array(strucGraph.graph.edges)[:,:2].T.shape[1]



In [ ]:
print('Timing results:')
print('_________________________________________________________')
print('Method | Time (s) | Number of edges')
print('_________________________________________________________')
print(f'CrystalNN | {stop_crystalNN-start_crystalNN:.2e} | {nEdges_crystalNN}')
print(f'MinimumDistanceNN | {stop_minDistNN-start_minDistNN:.2e} | {nEdges_minDistNN}')
# print(f'IsayevNN | {stop_isayevNN-start_isayevNN:.2e} | {nEdges_isayevNN}')
print(f'JmolNN | {stop_jmolNN-start_jmolNN:.2e} | {nEdges_jmolNN}')
print(f'MinimumOKeeffeNN | {stop_okeeffeNN-start_okeeffeNN:.2e} | {nEdges_okeeffeNN}')
# print(f'MinimumVIRENN | {stop_vireeNN-start_vireeNN:.2e} | {nEdges_vireeNN}')
# print(f'VoronoiNN | {stop_voronoiNN-start_voronoiNN:.2e} | {nEdges_voronoiNN}')
print('_________________________________________________________')

In [ ]:
start = timer()
supercell_dist = supercell.get_all_distances(mic=False)
supercell_atoms = supercell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(supercell)
crystal_nn = CrystalNN()
strucGraph = StructureGraph.with_local_env_strategy(struc, crystal_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = supercell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]].values
    for atom in supercell_atoms
    ], dtype='float')
node_pos_real = supercell.get_positions()
node_pos_relative = supercell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2f} s')
print(f'Number of edges: {len(direction[0])}')
#plot_atoms_and_bonds(node_pos_real, node_features, direction)

In [ ]:
start = timer()
# Get distances with MIC (NOTE I don't think this makes a difference as long as pbc=True in the unit cell)
supercell_dist = supercell.get_all_distances(mic=False)
supercell_atoms = supercell.get_atomic_numbers().reshape(-1, 1)

# metal_filter = np.isin(unit_cell_atoms, metals)

# Find node features
node_features = np.array([
    node_feature_table.loc[atom[0]-1].values
    for atom in supercell_atoms
    ], dtype='float')

# Create mask of threshold for bonds
bond_threshold = np.zeros_like(supercell_dist)
for i, r1 in enumerate(node_features[:,1]):
    bond_threshold[i,:] = (r1 + node_features[:,1]) / 100 * 1.25
np.fill_diagonal(bond_threshold, 0.)

# Find edges
direction = np.argwhere(supercell_dist < bond_threshold).T

# Handle case with no edges
if len(direction[0]) == 0:
    min_dist = np.amin(supercell_dist[supercell_dist > 0])
    direction = np.argwhere(supercell_dist < min_dist * 1.1).T
    
edge_features = supercell_dist[direction[0], direction[1]]

node_pos_real = supercell.get_positions()
node_pos_relative = supercell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2e} s')
print(f'Number of edges: {len(direction[0])}')
#plot_atoms_and_bonds(node_pos_real, node_features, direction, edge_features)

In [ ]:
len(direction[0])

In [ ]:
start = timer()
supercell_dist = supercell.get_all_distances(mic=False)
supercell_atoms = supercell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(supercell)
crystal_nn = CrystalNN(distance_cutoffs=None, x_diff_weight=0.0, porous_adjustment=False)
strucGraph = StructureGraph.with_local_env_strategy(struc, crystal_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = supercell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]].values
    for atom in supercell_atoms
    ], dtype='float')
node_pos_real = supercell.get_positions()
node_pos_relative = supercell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2f} s')
print(f'Number of edges: {len(direction[0])}')
#plot_atoms_and_bonds(node_pos_real, node_features, direction)

In [ ]:
start = timer()
supercell_dist = supercell.get_all_distances(mic=False)
supercell_atoms = supercell.get_atomic_numbers().reshape(-1, 1)

# Create edges and node features
struc = AseAtomsAdaptor.get_structure(supercell)
minDist_nn = MinimumDistanceNN()
strucGraph = StructureGraph.with_local_env_strategy(struc, minDist_nn)
direction = np.array(strucGraph.graph.edges)[:,:2].T

edge_features = supercell_dist[direction[0], direction[1]]
node_features = np.array([
    node_feature_table.loc[atom[0]].values
    for atom in supercell_atoms
    ], dtype='float')
node_pos_real = supercell.get_positions()
node_pos_relative = supercell.get_scaled_positions()

stop = timer()
print(f'Time to create graph: {stop-start:.2f} s')
print(f'Number of edges: {len(direction[0])}')
#plot_atoms_and_bonds(node_pos_real, node_features, direction)


## Misc

In [ ]:
from mendeleev import element
from mendeleev.fetch import fetch_table

In [ ]:
node_feature_table = fetch_table('elements')[['atomic_number', 'atomic_radius', 'atomic_weight', 'electron_affinity']]

In [ ]:
node_feature_table.describe()

In [ ]:
node_feature_table['electron_affinity'].fillna(0.0, inplace=True)

In [ ]:
node_feature_table['atomic_radius']

In [ ]:
node_feature_table.describe()

In [ ]:
import numpy as np

# Define the element symbol
element_symbol = 'O'

# Get the row corresponding to the element symbol
element_row = node_feature_table[node_feature_table['symbol'] == element_symbol]

# Extract the required features as a numpy array
element_features = element_row[['atomic_number', 'atomic_radius', 'atomic_weight', 'electron_affinity']].to_numpy()


In [ ]:
element_features.flatten()

# Load and inspect data

In [2]:
from Code.datasetClass import InOrgMatDatasets
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from mendeleev import element
from tqdm.auto import tqdm
warnings.simplefilter(action='ignore') # , category=FutureWarning

dataset = InOrgMatDatasets('Simulated_rmax60_v3', root='./Dataset/')

In [3]:
#import subprocess

# Define the command line commands
#commands = [
#    'python benchmarking.py --config ./benchmark_configs/config_GIN_nodeClassification.yml',
#    'python benchmarking.py --config ./benchmark_configs/config_EdgeCNN_nodeClassification.yml',
#    'python benchmarking.py --config ./benchmark_configs/config_AttentiveFP_nodeClassification.yml',
#    'python benchmarking.py --config ./benchmark_configs/config_GraphSAGE_nodeClassification.yml',
#    'python benchmarking.py --config ./benchmark_configs/config_GAT_nodeClassification.yml',
#    'python benchmarking.py --config ./benchmark_configs/config_GCN_nodeClassification.yml',
#]

# Run the commands
#for command in commands:
#    subprocess.run(command, shell=True)


In [4]:
dataset.create_data_split()
# dataset.create_data_split(split_strategy='stratified', stratify_on='Space group (Number)')

In [3]:
dataset.load_data_split(split_strategy='random', stratify_on='Space group (Number)')

In [4]:
idx = 1
print(dataset.len())
print(dataset.get(idx).y['atomic_species'])
print(dataset.get(idx))

3180
tensor([ 8., 20.])
Data(
  x=[279, 4],
  edge_index=[2, 1200],
  edge_attr=[1200],
  y={
    crystal_type='CadmiumChloride',
    space_group_symbol='C 2/m',
    space_group_number=12,
    atomic_species=[2],
    n_atomic_species=2,
    np_size=23.96804428100586,
    n_atoms=279,
    n_bonds=1200,
    cell_params=[6],
    unit_cell_node_feat=[9, 4],
    unit_cell_edge_index=[2, 12],
    unit_cell_edge_feat=[12],
    unit_cell_pos_abs=[9, 3],
    unit_cell_pos_frac=[9, 3],
    nd=[2, 580],
    xrd=[2, 580],
    nPDF=[2, 6000],
    xPDF=[2, 6000],
    sans=[2, 300],
    saxs=[2, 300],
  },
  pos_frac=[279, 3],
  pos_abs=[279, 3]
)


In [4]:
print(dataset.len(split='train'), dataset.len(split='validation'), dataset.len(split='test'))
dataset.get(1, split='test')

88641 11081 11081


Data(
  x=[47, 4],
  edge_index=[2, 108],
  edge_attr=[47],
  y={
    crystal_type='Unknown',
    space_group_symbol='I 41/a m d',
    space_group_number=141,
    atomic_species=[3],
    n_atomic_species=3,
    np_size=11.920214653015137,
    n_atoms=47,
    n_bonds=108,
    cell_params=[6],
    unit_cell_node_feat=[24, 4],
    unit_cell_edge_index=[2, 112],
    unit_cell_edge_feat=[112],
    unit_cell_pos_abs=[24, 3],
    unit_cell_pos_frac=[24, 3],
    nd=[2, 580],
    xrd=[2, 580],
    nPDF=[2, 6000],
    xPDF=[2, 6000],
    sans=[2, 300],
    saxs=[2, 300],
  },
  pos_frac=[47, 3],
  pos_abs=[47, 3]
)

## Create .xyz file

In [ ]:
# Loop over all indices in the dataset
for idx in tqdm(range(dataset.len())):
    # Get the sample at an index
    # idx = 9
    sample = dataset.get(idx)

    # Extract the absolute coordinates and atomic numbers
    coords = sample.pos_abs.numpy()
    atomic_numbers = sample.x[:,0].numpy().astype(int)
    n_atoms = sample.y['n_atoms']

    # Write the .xyz file
    with open(f'./Dataset/xyz/SimulatedTest2/{idx}.xyz', 'w') as f:
        f.write(f'{n_atoms}\n')
        f.write('\n')
        for i in range(n_atoms):
            f.write(f'{element(int(atomic_numbers[i])).symbol} {coords[i,0]} {coords[i,1]} {coords[i,2]}\n')


## Plotting and statistics

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(*dataset.get(540, split='train').y['xPDF'])
plt.ylabel('Intensity (a.u.)')
plt.xlabel('r ($\AA$)')
plt.show()

In [5]:
stats = dataset.get_statistics(return_dataframe=True)
stats.head()

,idx,# of nodes,# of edges,# of elements,Space group (Symbol),Space group (Number),Crystal type,NP size (Å),Elements,Random data split
0,0,31.0,108.0,2.0,C 2/m,12.0,CadmiumChloride,10.817969,"[tensor(8.), tensor(20.)]",Test
1,1,279.0,1200.0,2.0,C 2/m,12.0,CadmiumChloride,23.968044,"[tensor(8.), tensor(20.)]",Train
2,2,875.0,4188.0,2.0,C 2/m,12.0,CadmiumChloride,34.560139,"[tensor(8.), tensor(20.)]",Train
3,3,2005.0,10188.0,2.0,C 2/m,12.0,CadmiumChloride,43.835541,"[tensor(8.), tensor(20.)]",Train
4,4,3927.0,20604.0,2.0,C 2/m,12.0,CadmiumChloride,54.064667,"[tensor(8.), tensor(20.)]",Train


In [6]:
stats.describe()

,idx,# of nodes,# of edges,# of elements,Space group (Number),NP size (Å)
count,3180.000000,3180.000000,3180.000000,3180.0,3180.000000,3180.000000
mean,1589.500000,2188.391509,15605.169811,2.0,151.583333,33.304232
std,918.131254,2319.871145,19372.138747,0.0,89.485164,14.699663
min,0.000000,7.000000,7.000000,2.0,12.000000,7.363739
25%,794.750000,367.000000,1560.000000,2.0,56.250000,22.601038
50%,1589.500000,1377.000000,7212.000000,2.0,218.500000,33.498924
75%,2384.250000,3428.000000,23712.000000,2.0,225.000000,44.043523
max,3179.000000,14793.000000,118258.000000,2.0,227.000000,56.690269


In [7]:
stats[stats['# of edges'] == 0]

,idx,# of nodes,# of edges,# of elements,Space group (Symbol),Space group (Number),Crystal type,NP size (Å),Elements,Random data split


In [ ]:
sns.scatterplot(data=stats, x='# of nodes', y='# of edges', hue='Crystal type')
plt.show()

In [ ]:
palette = sns.color_palette('tab10')
color_dict = {'Train': palette[0], 'Validation': palette[1], 'Test': palette[2]}
hue_order = ['Train', 'Validation', 'Test']

In [ ]:
plt.figure(figsize=(15,5))
ax = sns.histplot(data=stats, x='NP size (Å)', binwidth=0.5, hue='Random data split', multiple='layer',kde=True, kde_kws=dict(bw_method=0.1), line_kws=dict(linewidth=2), stat='count', palette=color_dict, hue_order=hue_order)
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_sizeDist.png', format='png', dpi=300)
plt.show()

plt.figure(figsize=(15,5))
ax = sns.histplot(data=stats, x='NP size (Å)', binwidth=0.5, hue='Stratified data split (Crystal type)', multiple='layer',kde=True, kde_kws=dict(bw_method=0.1), line_kws=dict(linewidth=2), stat='count', palette=color_dict, hue_order=hue_order)
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_sizeDist.png', format='png', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.histplot(data=stats, x='Crystal type', hue='Random data split', multiple='dodge', palette=color_dict, hue_order=hue_order)
# ax = sns.histplot(data=stats, x='Crystal type', hue='Stratified data split (Crystal type)', multiple='dodge', palette=color_dict, hue_order=hue_order)
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_crystalTypes.png', format='png', dpi=300)
plt.show()

plt.figure(figsize=(10,5))
#ax = sns.histplot(data=stats, x='Crystal type', hue='Random data split', multiple='dodge', palette=color_dict, hue_order=hue_order)
ax = sns.histplot(data=stats, x='Crystal type', hue='Stratified data split (Crystal type)', multiple='dodge', palette=color_dict, hue_order=hue_order)
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_crystalTypes.png', format='png', dpi=300)
plt.show()

In [ ]:
ax = sns.pairplot(data=stats.drop('Elements', axis=1), hue='Random data split', diag_kws=dict(bw_method=0.1), palette=color_dict, hue_order=hue_order)
# ax = sns.pairplot(data=stats.drop('Elements', axis=1), hue='Stratified data split (Crystal type)', diag_kws=dict(bw_method=0.1), palette=color_dict, hue_order=hue_order)
# ax = sns.pairplot(data=stats.drop('Elements', axis=1), diag_kws=dict(bw_method=0.1))
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_pairplot.png', format='png', dpi=300)
plt.show()

#ax = sns.pairplot(data=stats, hue='Random data split', diag_kws=dict(bw_method=0.1))
ax = sns.pairplot(data=stats.drop('Elements', axis=1), hue='Stratified data split (Crystal type)', diag_kws=dict(bw_method=0.1), palette=color_dict, hue_order=hue_order)
# ax = sns.pairplot(data=stats.drop('Elements', axis=1), diag_kws=dict(bw_method=0.1))
sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.tight_layout()
# plt.savefig('./test_stats_pairplot.png', format='png', dpi=300)
plt.show()

In [ ]:
stats.columns